In [68]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from openml import datasets

openml_datasets_index = pd.read_csv('openml_datasets_index.csv')
uci_datasets_index = pd.read_csv('uci_datasets_index.csv')

openml_index = openml_datasets_index
uci_index = uci_datasets_index

In [69]:
from math import ceil
openml_index = openml_datasets_index
uci_index = uci_datasets_index


openml_index = openml_index[openml_index['NumberOfClasses'] == 2]
openml_index = openml_index[openml_index['NumberOfMissingValues'] == 0]
openml_index = openml_index[openml_index['NumberOfInstancesWithMissingValues'] == 0]

passing_dids = []
for _row in openml_index.iterrows():
    row = _row[1]
    
    dataset_size = row['NumberOfInstances']
    pos_prop = row['MajorityClassSize'] / dataset_size
    neg_prop = row['MinorityClassSize'] / dataset_size

    sample_size = 100
    test_size = 0.5

    pos_number_sample_test = ceil(pos_prop * dataset_size * test_size)
    neg_number_sample_test = ceil(neg_prop * dataset_size * test_size)

    if pos_number_sample_test > sample_size and neg_number_sample_test > sample_size:
        passing_dids.append(row['did'])

openml_index = openml_index[openml_index['did'].isin(passing_dids)]
print(len(passing_dids))

908


In [70]:
openml_index

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
1,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0
26,31,credit-g,1,1,active,ARFF,700.0,10.0,300.0,2.0,21.0,1000.0,0.0,0.0,7.0,14.0
31,37,diabetes,1,1,active,ARFF,500.0,2.0,268.0,2.0,9.0,768.0,0.0,0.0,8.0,1.0
38,44,spambase,1,1,active,ARFF,2788.0,2.0,1813.0,2.0,58.0,4601.0,0.0,0.0,57.0,1.0
42,50,tic-tac-toe,1,1,active,ARFF,626.0,3.0,332.0,2.0,10.0,958.0,0.0,0.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860,46553,Loan_Status,1,45956,active,arff,35000.0,NaN,10000.0,2.0,14.0,45000.0,0.0,0.0,8.0,5.0
5861,46554,Loan_Status,2,45956,active,arff,35000.0,NaN,10000.0,2.0,14.0,45000.0,0.0,0.0,8.0,6.0
5869,46562,German-Credit-Data-Creditability-Preprocessed-...,1,45575,active,arff,700.0,NaN,300.0,2.0,34.0,1000.0,0.0,0.0,33.0,1.0
5870,46563,Loan_Approval_Status_Classification,2,45956,active,arff,35000.0,NaN,10000.0,2.0,14.0,45000.0,0.0,0.0,13.0,1.0


In [71]:
did = openml_index['did'].tolist()[1]

dataset = datasets.get_dataset(int(did))

dataset_name = dataset.name
X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format="dataframe", target=dataset.default_target_attribute
)

In [72]:
categorical_indicator

[True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True]

In [73]:
list = [True, False, False]
True in list

True

In [79]:
categorical_indicator

[True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True]

In [80]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

preprocessing_log = pd.DataFrame(columns=['Dataset', 'OneHotEncoder', 'LabelEncoder'])

def preprocess_data(X, y, dataset_name, categorical_indicator, preprocessing_log):
    one_hot_encoder_used = False
    label_encoder_used = False
    
    if any(categorical_indicator):
        encoder = OneHotEncoder(sparse_output=False, drop='first')
        X_encoded = encoder.fit_transform(X.loc[:, categorical_indicator])
        X_non_categorical = X.loc[:, ~np.array(categorical_indicator)].values
        X_processed = np.hstack((X_non_categorical, X_encoded))
        one_hot_encoder_used = True
    else:
        X_processed = X.values
    
    if y.dtype == 'object' or y.dtype.name == 'category':
        label_encoder = LabelEncoder()
        y_processed = label_encoder.fit_transform(y)
        label_encoder_used = True
    else:
        y_processed = y
    
    preprocessing_log.loc[len(preprocessing_log.index)] = [dataset_name, 'Yes' if one_hot_encoder_used else 'No', 'Yes' if label_encoder_used else 'No']
    
    return X_processed, y_processed, preprocessing_log

X_processed, y_processed, preprocessing_log = preprocess_data(X, y, dataset_name, categorical_indicator, preprocessing_log)

In [81]:
preprocessing_log

,Dataset,OneHotEncoder,LabelEncoder
0,credit-g,Yes,Yes


In [20]:
X

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4,male single,none,4,real estate,67,none,own,2,skilled,1,yes,yes
1,0<=X<200,48,existing paid,radio/tv,5951.0,<100,1<=X<4,2,female div/dep/mar,none,2,real estate,22,none,own,1,skilled,1,none,yes
2,no checking,12,critical/other existing credit,education,2096.0,<100,4<=X<7,2,male single,none,3,real estate,49,none,own,1,unskilled resident,2,none,yes
3,<0,42,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2,male single,guarantor,4,life insurance,45,none,for free,1,skilled,2,none,yes
4,<0,24,delayed previously,new car,4870.0,<100,1<=X<4,3,male single,none,4,no known property,53,none,for free,2,skilled,2,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12,existing paid,furniture/equipment,1736.0,<100,4<=X<7,3,female div/dep/mar,none,4,real estate,31,none,own,1,unskilled resident,1,none,yes
996,<0,30,existing paid,used car,3857.0,<100,1<=X<4,4,male div/sep,none,4,life insurance,40,none,own,1,high qualif/self emp/mgmt,1,yes,yes
997,no checking,12,existing paid,radio/tv,804.0,<100,>=7,4,male single,none,4,car,38,none,own,1,skilled,1,none,yes
998,<0,45,existing paid,radio/tv,1845.0,<100,1<=X<4,4,male single,none,4,no known property,23,none,for free,1,skilled,1,yes,yes


In [ ]:

X['class']